# Fish disease diagnosis with MobileNet

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Setting up

In [2]:
%pip install tensorflow==2.16.0rc0 matplotlib seaborn scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 62.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 66.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 63.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 61.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 36.0 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.9.0
    Uninstalling h5py-3.9.0:
      Successfully uninstalled h5py-3.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.2
    Uninstalling tensorboard-2.15.2:
      Successfully uninstalled tensorboard-2.15.2
  Attempting uninstall: keras
    Fo

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Sequential, callbacks, losses, optimizers, metrics
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras import backend as K
from tensorflow.keras.saving import save_model, load_model
import seaborn as sns
from sklearn import metrics
from tensorflow.keras.models import load_model

## Configuration the parameters

In [4]:
input_shape = (224, 224, 3)
num_classes = 2

## Loading and pre-processing dataset

In [5]:
train_ds_path = '/content/drive/MyDrive/EfficientNetB0/data_xray/chest_xray/train'
val_ds_path = '/content/drive/MyDrive/EfficientNetB0/data_xray/chest_xray/val'
test_ds_path = '/content/drive/MyDrive/EfficientNetB0/data_xray/chest_xray/test'

In [6]:
from tensorflow.keras.preprocessing import image_dataset_from_directory

train_ds = image_dataset_from_directory(
    train_ds_path,
    labels="inferred",
    label_mode="int",
    color_mode="rgb",
    batch_size=32,
    image_size=(input_shape[0], input_shape[1]),
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset="training"
)

val_ds = image_dataset_from_directory(
    train_ds_path,
    labels="inferred",
    label_mode="int",
    color_mode="rgb",
    batch_size=32,
    image_size=(input_shape[0], input_shape[1]),
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset="validation"
)


Found 5216 files belonging to 2 classes.
Using 4173 files for training.
Found 5216 files belonging to 2 classes.
Using 1043 files for validation.


In [7]:
data_preprocessing = keras.Sequential(
    [
        layers.Normalization(),
    ],
    name="data_preprocessing",
)

# Compute the mean and the variance of the training data for normalization.
unbatch_train_ds = train_ds.unbatch()
feature_ds = unbatch_train_ds.map(lambda x, y: x)
data_preprocessing.layers[0].adapt(feature_ds)

KeyboardInterrupt: 

## Use data augmentation

In [ ]:
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(factor=0.02),
        layers.RandomZoom(height_factor=0.2, width_factor=0.2),
    ],
    name="data_augmentation",
)

## The MobileNet Model

In [ ]:
def create_mobilenet_classifier(optimizer, learning_rate, drop_out, weight_decay):
  # Load a pre-trained model
  pretrained_model = tf.keras.applications.MobileNet(
      input_shape=input_shape,
      include_top=False,
      weights="imagenet"
  )

  # Implement transfer learning
  pretrained_model.trainable = False

  model = Sequential([
      data_preprocessing,
      data_augmentation,
      pretrained_model,
      layers.GlobalAveragePooling2D(keepdims=True),
      layers.Dropout(drop_out, name='dropout'),
      layers.Conv2D(2, (1, 1), padding='same'),
      layers.Reshape((2,), name='reshape_2'),
      layers.Activation(activation='softmax')
  ])

  optimizer = optimizers.get(optimizer)
  optimizer.learning_rate = learning_rate
  optimizer.weight_decay = weight_decay

  loss = losses.SparseCategoricalCrossentropy()

  model.compile(
      optimizer=optimizer,
      loss=loss,
      metrics=['accuracy']
  )

  return model

## Hyperparameters tuning

In [ ]:
num_epochs = 1000


In [ ]:
model = create_mobilenet_classifier('adam', 0.001, 0.3, 0.0001)
history = model.fit(
      train_ds,
      epochs=30,
      verbose=0,
      callbacks=callbacks.EarlyStopping(
          monitor="val_loss", min_delta=0, patience=5, verbose=1,
          baseline=True, restore_best_weights=True, start_from_epoch=10
      ),
      validation_data=val_ds,
      shuffle=False,
  )

## Compile, train and evaluate the model with best hyperparameters

In [ ]:
!rm -r logs
!mkdir logs

In [ ]:
csv_logger_filepath = 'logs/result.csv'
save_model_filpath = 'logs/mobilenet.keras'

In [ ]:
# Call back
csv_logger = callbacks.CSVLogger(
    csv_logger_filepath,
    separator=",",
    append=True
)

early_stopping = keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=5,
    verbose=1,
    mode="auto",
    baseline=True,
    restore_best_weights=True,
    start_from_epoch=10,
)

def run_experiment(model, batch_size, num_epochs):

    train_ds.batch_size = batch_size
    val_ds.batch_size = batch_size
    test_ds.batch_size = batch_size

    history = model.fit(
        train_ds,
        epochs=num_epochs,
        callbacks=[
            csv_logger, early_stopping
        ],
        validation_data=val_ds,
        shuffle=False,
    )

    save_model(model, save_model_filpath, overwrite=True)

    # Evaluate the model
    metric_results = model.evaluate(test_ds, return_dict=True )
    print(f"Test accuracy: {round(metric_results['accuracy'] * 100, 2)}%")


    return history, model

In [ ]:
print(best_learning_rate)
print(best_optimizer)
print(best_dropout)
print(best_batch_size)
print(best_weight_decay)

In [ ]:
model = create_mobilenet_classifier(best_learning_rate,best_optimizer, best_dropout, best_weight_decay)
history, model = run_experiment(model, best_batch_size, num_epochs)

In [ ]:
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

train_loss = history.history['loss']
val_loss = history.history['val_loss']

unbatch_test_ds = test_ds.unbatch()
true_labels = list(unbatch_test_ds.map(lambda x, y: y))
predicted_labels = model.predict(test_ds).argmax(axis=-1)

print(predicted_labels)

In [ ]:
plt.axis(ymin=0.0, ymax=1)
plt.plot(range(1, len(train_acc) + 1), train_acc, label='Training Accuracy')
plt.plot(range(1, len(val_acc) + 1), val_acc, label='Validation Accuracy')
plt.title('Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.savefig('logs/accuracy.png')
plt.show()

plt.axis(ymin=0.0, ymax=1)
plt.plot(range(1, len(train_loss) + 1), train_loss, label='Training Loss')
plt.plot(range(1, len(val_loss) + 1), val_loss, label='Validation Loss')
plt.title('Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.savefig('logs/loss.png')
plt.show()

report = metrics.classification_report(true_labels,predicted_labels, digits=4)
print(report)

cm = tf.math.confusion_matrix(labels=true_labels, predictions=predicted_labels).numpy()
ncm = np.round(cm/np.sum(cm, axis=1).reshape(-1,1),2)
fig, ax = plt.subplots(figsize=(12, 8))
ax = sns.heatmap(ncm, annot=True, cmap='Blues', fmt='g', annot_kws={"size":15})
ax.set_title('Normalized confusion matrix\n\n', fontsize=15);
ax.set_xlabel('\nPredicted label', fontsize=15)
ax.set_ylabel('True label ', fontsize=15);
ax.xaxis.set_ticklabels(['virus','normal'], fontsize=13)
ax.yaxis.set_ticklabels(['virus','normal'], fontsize=13)
plt.savefig('logs/normalized_confusion_atrix.png')
plt.show()

In [ ]:
!rm -f logs.zip
!zip -r logs.zip logs